In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [59]:
import os 
from pinecone import Pinecone , ServerlessSpec
index_name = "hybrid-search-langchain"
from dotenv import load_dotenv
import pinecone
load_dotenv()

True

In [60]:
pc = Pinecone(api_key=os.getenv("pinecone_api_key"))

In [63]:
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,dimension=768 , metric='dotproduct' , spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [64]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000241000FEB90>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None)

In [53]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

In [65]:
sentences = [
    "in 2023 , I visited Paris" , 
    "In 2022 , I visited London" ,
    "In 2021 , I visited New York"
]
bm25_encoder.fit(sentences)
bm25_encoder.dump("bm25_vals.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [66]:
retriever = PineconeHybridSearchRetriever(index=index , embeddings=embeddings , sparse_encoder=bm25_encoder)

In [67]:
retriever

PineconeHybridSearchRetriever(embeddings=GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000241000FEB90>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000002417BD0E390>, index=<pinecone.data.index.Index object at 0x00000241000FCF10>)

In [68]:
retriever.add_texts(sentences)

  0%|          | 0/1 [00:00<?, ?it/s]

In [69]:
retriever.invoke("which city did i visit in 2022 ?")

[Document(page_content='In 2022 , I visited London'),
 Document(page_content='in 2023 , I visited Paris'),
 Document(page_content='In 2021 , I visited New York')]